<div style="display: flex; align-items: center; justify-content: space-between; border-bottom: 3px solid #D0D0D0; padding: 10px; font-family: Arial, sans-serif;">
  <!-- Logo -->
  <img src="https://upload.wikimedia.org/wikipedia/commons/1/1c/Uoc_masterbrand_vertical.png" alt="Logo" style="width: 150px; height: auto;">

  <!-- Detalles -->
  <div style="text-align: right; line-height: 1.6; margin-left: 20px; margin-right: 80px">
    <p style="margin: 0; font-size: 1.2em; font-weight: bold; color: #333;">Proyecto: <span style="color: #878787;">Práctica - Parte 2</span></p>
    <p style="margin: 0; font-size: 1em; font-weight: bold; color: #333;">Notebook: <span style="color: #878787;">Exploración inicial de datos de </span></p>
    <p style="color: #878787;">Juan Luis González Rodríguez - Visualización de datos</span></p>
  </div>
</div>

# 0. Librerías

In [64]:
import pandas as pd
import csv

# 1. Carga de dataframes

Se cargan los datos a unir de los diferentes dataframes para crear la visualización

In [65]:
# Este dataset contiene los datos de los inclinómetros con los valores imputados
# Para los intervalos de tiempo en los que estaba apagado el dispositivo
# menos de 6 segundos
paht_inclinometer = '../0_data/clean_MMASH/inclinometer_clean.pkl'
df_incl = pd.read_pickle(paht_inclinometer)

# Se deshace el one-hot encoding del inclinómetro
df_incl['Inclinometer'] = df_incl[['Inclinometer Off', 'Inclinometer Standing', 'Inclinometer Sitting', 'Inclinometer Lying']].idxmax(axis=1)
df_incl.drop(columns=['Inclinometer Off', 'Inclinometer Standing', 'Inclinometer Sitting', 'Inclinometer Lying'], inplace=True)
# Se traduce el inclinómetro a español
esp_mapper = {
    'Inclinometer Standing': 'De pie',
    'Inclinometer Sitting': 'Sentado',
    'Inclinometer Lying': 'Tumbado',
    'Inclinometer Off': 'Apagado'
}
df_incl['Inclinometer'] = df_incl['Inclinometer'].map(esp_mapper)


display(df_incl.head())
print(f"Shape: {df_incl.shape}")

,datetime,User,Inclinometer
0,1900-01-01 10:10:22,user_1,De pie
1,1900-01-01 10:10:23,user_1,De pie
2,1900-01-01 10:10:24,user_1,Sentado
3,1900-01-01 10:10:25,user_1,Sentado
4,1900-01-01 10:10:26,user_1,De pie


Shape: (1905665, 3)


In [66]:
# Este dataset contiene los datos de la frecuencia cardíaca recogidas por el mismo dispositivo que el inclinómetro
# a los que se le ha aplicado una media movil para mejorar la visualización y corregir errores extremos
path_bpm = '../0_data/clean_MMASH/actigraph_clean.pkl'
df_bpm = pd.read_pickle(path_bpm)

# Se seleccionan las columnas de interés
df_bpm = df_bpm[['datetime', 'User', 'HR']]

# Se redondean la columna de HR a 1 decimal
df_bpm['HR'] = df_bpm['HR'].round(1)

display(df_bpm.head())
df_bpm.shape

,datetime,User,HR
0,1900-01-01 10:10:22,user_1,93.0
1,1900-01-01 10:10:23,user_1,93.0
2,1900-01-01 10:10:24,user_1,93.0
3,1900-01-01 10:10:25,user_1,93.0
4,1900-01-01 10:10:26,user_1,93.0


(1462678, 3)

In [67]:
print(f"El dataset de los inclinómetros tiene {df_incl.shape[0] - df_bpm.shape[0]} filas más que el de la frecuencia cardíaca")

El dataset de los inclinómetros tiene 442987 filas más que el de la frecuencia cardíaca


Se observa que el dataset de los inclinómetros tiene más filas que el de frecuencia cardíaca. Esto se debe a que el dataset de inclinómetros tiene 1 fila por cada segundo, ya que en el proceso de limpieza se imputaron estos valores faltantes. Por otro lado, la frecuencia cardíaca no se imputó, por lo que solo se tienen los valores que se recogieron.

En base a esto se hará una unión de los dos dataframes. Se hará  un left join con el dataframe de los inclinómetros, ya que este tiene más filas que el de la frecuencia cardíaca. Para los registros que no tengan una frecuencia cardíaca asociada se registrará un NaN en la columna de HR.

In [68]:
df_vis = pd.merge(df_incl, df_bpm, on=['datetime', 'User'], how='left')

print(f"El dataset final tiene {df_vis.shape[0]} filas y {df_vis.shape[1]} columnas")

El dataset final tiene 1905665 filas y 4 columnas


In [69]:
# Mostramos cuantos NA hay en cada columna
display(df_vis.isna().sum())

datetime             0
User                 0
Inclinometer         0
HR              442987
dtype: int64

In [70]:
df_vis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1905665 entries, 0 to 1905664
Data columns (total 4 columns):
 #   Column        Dtype         
---  ------        -----         
 0   datetime      datetime64[ns]
 1   User          object        
 2   Inclinometer  object        
 3   HR            float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 58.2+ MB


El valor de los NA en la columna de HR es igual que la diferencia entre el número de filas de los dos datasets. Por lo que se confirma la correcta unión de los dos datasets.

Se cambia el nombre de los usuarios para que sea más fácil de leer

In [ ]:
def pretty_user_text(user):
    """
    Esta función formatea el texto del identificador de un usuario para que sea más legible.
    
    Args:
    user: Identificador del usuario.
    
    Returns:
    Identificador del usuario formateado.
    """
    user = user.replace('user_', 'Usuario ')
    return user

# Se formatea el texto de los usuarios
df_vis['User'] = df_vis['User'].apply(pretty_user_text)

# Guardar los datos preparados para la visualización

In [71]:
# Se guardan los datos en un archivo csv
path_vis_data = './visualizacion_data.csv'
df_vis.to_csv(path_vis_data, index=False, quotechar='"', sep=',', encoding='utf-8', quoting=csv.QUOTE_NONNUMERIC)